# Import Libaries

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
data = pd.read_feather('output.feather')
data

,abaqus,ablation,ableton live,abnormal psychology,absorption,abstraction,abutments,ac motors,academic advising,academic affairs,...,zemax,zendesk,zeta potential,zipper,zone,zoning,zoology,zoom,zumba,zumba fitness
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
resume = pd.read_csv('Resume.csv')
y = resume['Category']
data['y'] = y
data

,abaqus,ablation,ableton live,abnormal psychology,absorption,abstraction,abutments,ac motors,academic advising,academic affairs,...,zendesk,zeta potential,zipper,zone,zoning,zoology,zoom,zumba,zumba fitness,y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION


# Aggregation by Class

In [4]:
agg_data = data.groupby(['y']).sum()
agg_data

,abaqus,ablation,ableton live,abnormal psychology,absorption,abstraction,abutments,ac motors,academic advising,academic affairs,...,zemax,zendesk,zeta potential,zipper,zone,zoning,zoology,zoom,zumba,zumba fitness
y,,,,,,,,,,,,,,,,,,,,,
ACCOUNTANT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
ADVOCATE,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
AGRICULTURE,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
APPAREL,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,1,3,0,0,0,0
ARTS,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,1,0
AUTOMOBILE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AVIATION,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
BANKING,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
BPO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
agg_data = agg_data.T
agg_data

y,ACCOUNTANT,ADVOCATE,AGRICULTURE,APPAREL,ARTS,AUTOMOBILE,AVIATION,BANKING,BPO,BUSINESS-DEVELOPMENT,...,DIGITAL-MEDIA,ENGINEERING,FINANCE,FITNESS,HEALTHCARE,HR,INFORMATION-TECHNOLOGY,PUBLIC-RELATIONS,SALES,TEACHER
abaqus,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
ablation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
ableton live,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
abnormal psychology,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
absorption,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zoning,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
zoology,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoom,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
zumba,0,0,0,0,1,0,0,0,0,0,...,0,0,0,6,0,0,0,0,0,0


In [6]:
agg_data.describe()

y,ACCOUNTANT,ADVOCATE,AGRICULTURE,APPAREL,ARTS,AUTOMOBILE,AVIATION,BANKING,BPO,BUSINESS-DEVELOPMENT,...,DIGITAL-MEDIA,ENGINEERING,FINANCE,FITNESS,HEALTHCARE,HR,INFORMATION-TECHNOLOGY,PUBLIC-RELATIONS,SALES,TEACHER
count,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,...,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000,8162.000000
mean,0.590909,0.509556,0.307033,0.419995,0.440333,0.161112,0.552438,0.563465,0.125337,0.579392,...,0.514457,0.697133,0.599975,0.449032,0.579882,0.505758,0.783631,0.595687,0.428204,0.331536
std,3.914398,2.785246,1.462457,2.554737,2.297275,0.871694,2.964687,3.282657,0.679020,3.657548,...,2.770360,2.924850,3.742688,2.700490,2.908236,3.172225,3.790234,3.755676,2.960393,2.279106
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,102.000000,100.000000,54.000000,85.000000,79.000000,30.000000,97.000000,106.000000,22.000000,119.000000,...,79.000000,97.000000,118.000000,96.000000,107.000000,104.000000,112.000000,106.000000,116.000000,76.000000


# Identify Significant Skills

We used percentile method here, any value smaller than 90 percentile is not significant. 

In [7]:
# We want to find among all the skills that appear at least once, what is the 95 percentile value for a particular job
print("Accountant:", np.percentile(agg_data['ACCOUNTANT'][agg_data['ACCOUNTANT'] != 0], 95))
print("BPO:", np.percentile(agg_data['BPO'][agg_data['BPO'] != 0], 95))

Accountant: 21.0
BPO: 5.0


In [8]:
# Accountant significant skills
temp = agg_data['ACCOUNTANT'][agg_data['ACCOUNTANT'] > np.percentile(agg_data['ACCOUNTANT'][agg_data['ACCOUNTANT'] != 0], 95)]
temp

audit                       62
auditing                    34
balance                     58
balance sheet               43
bank reconciliations        34
banking                     29
billing                     49
bookkeeping                 27
budget                      71
budgeting                   33
business administration     50
cash flow                   30
close                       42
closing                     55
collections                 26
communication               51
customer service            29
depreciation                24
detail oriented             22
filing                      31
finance                     62
financial analysis          24
financial statement         34
financial statements        74
forecasting                 28
general ledger              77
invoice                     30
invoicing                   23
journal                     72
leadership                  24
ledger                      83
level                       26
manageme

In [9]:
print(temp.max(), temp.min())

102 22


In [10]:
# Scaling
temp.apply(lambda x: (x - temp.min()) / (temp.max() - temp.min()))

audit                      0.5000
auditing                   0.1500
balance                    0.4500
balance sheet              0.2625
bank reconciliations       0.1500
banking                    0.0875
billing                    0.3375
bookkeeping                0.0625
budget                     0.6125
budgeting                  0.1375
business administration    0.3500
cash flow                  0.1000
close                      0.2500
closing                    0.4125
collections                0.0500
communication              0.3625
customer service           0.0875
depreciation               0.0250
detail oriented            0.0000
filing                     0.1125
finance                    0.5000
financial analysis         0.0250
financial statement        0.1500
financial statements       0.6500
forecasting                0.0750
general ledger             0.6875
invoice                    0.1000
invoicing                  0.0125
journal                    0.6250
leadership    

In [11]:
# If value > 0.7, skill level required is 5; 
# If 0.7 >= value > 0.3, skill level required is 4;
# If 0.3 >= value, skill level required is 3;
temp2 = temp.apply(lambda x: (x - temp.min()) / (temp.max() - temp.min())).apply(lambda x: 5 if x > 0.7 else 4 if x > 0.3 else 3)
temp2

audit                      4
auditing                   3
balance                    4
balance sheet              3
bank reconciliations       3
banking                    3
billing                    4
bookkeeping                3
budget                     4
budgeting                  3
business administration    4
cash flow                  3
close                      3
closing                    4
collections                3
communication              4
customer service           3
depreciation               3
detail oriented            3
filing                     3
finance                    4
financial analysis         3
financial statement        3
financial statements       4
forecasting                3
general ledger             4
invoice                    3
invoicing                  3
journal                    4
leadership                 3
ledger                     5
level                      3
management                 5
microsoft office           4
operations    

In [12]:
# show the advanced skills to have so that you are better than most of your peers
temp2[temp2 == 5]

ledger        5
management    5
Name: ACCOUNTANT, dtype: int64

In [14]:
skills_required = {}
for col in agg_data.columns:
    print(col)
    series = agg_data[col][agg_data[col] > np.percentile(agg_data[col][agg_data[col] != 0], 95)]
    scaled_series = series.apply(lambda x: (x - series.min()) / (series.max() - series.min()))
    binned_series = scaled_series.apply(lambda x: 5 if x > 0.7 else 4 if x > 0.3 else 3)
    skills_required[col] = binned_series.to_dict()

ACCOUNTANT
ADVOCATE
AGRICULTURE
APPAREL
ARTS
AUTOMOBILE
AVIATION
BANKING
BPO
BUSINESS-DEVELOPMENT
CHEF
CONSTRUCTION
CONSULTANT
DESIGNER
DIGITAL-MEDIA
ENGINEERING
FINANCE
FITNESS
HEALTHCARE
HR
INFORMATION-TECHNOLOGY
PUBLIC-RELATIONS
SALES
TEACHER


In [15]:
skills_required

{'ACCOUNTANT': {'audit': 4,
  'auditing': 3,
  'balance': 4,
  'balance sheet': 3,
  'bank reconciliations': 3,
  'banking': 3,
  'billing': 4,
  'bookkeeping': 3,
  'budget': 4,
  'budgeting': 3,
  'business administration': 4,
  'cash flow': 3,
  'close': 3,
  'closing': 4,
  'collections': 3,
  'communication': 4,
  'customer service': 3,
  'depreciation': 3,
  'detail oriented': 3,
  'filing': 3,
  'finance': 4,
  'financial analysis': 3,
  'financial statement': 3,
  'financial statements': 4,
  'forecasting': 3,
  'general ledger': 4,
  'invoice': 3,
  'invoicing': 3,
  'journal': 4,
  'leadership': 3,
  'ledger': 5,
  'level': 3,
  'management': 5,
  'microsoft office': 4,
  'operations': 4,
  'plan': 3,
  'planning': 3,
  'problem solving': 3,
  'purchase': 3,
  'quickbooks': 4,
  'r': 3,
  'reconciliation': 4,
  'research': 3,
  'sales': 4,
  'sales tax': 3,
  'spreadsheets': 3,
  'tax returns': 3,
  'time management': 3,
  'track': 3},
 'ADVOCATE': {'active listening': 3,
  '

In [16]:
skills_file = open('resume_skills_required.pkl', 'wb')
pickle.dump(skills_required, skills_file)
skills_file.close()

In [2]:
with open('resume_skills_required', 'rb') as pickle_file:
    content = pickle.load(pickle_file)